SOLO AÑADIR COSAS DEBAJO DEL CHUNK DE CADA NOMBRE

ESTA PARTE ES PARA JAVIER

ESTA PARTE ES PARA ALMODOVAR

ESTA PARTE ES PARA ALBERTO

In [4]:
# 2 - Average total price per trip excluding taxes
import pandas as pd
import time
start = time.time()
data1=pd.read_csv("C:/Users/Alberto/Desktop/Máster UC3M/3rd quarter/Computación Escalada/SDComputing_labs/LAB2/tripdata_2017-01.csv",header=0)
data2=pd.read_csv("C:/Users/Alberto/Desktop/Máster UC3M/3rd quarter/Computación Escalada/SDComputing_labs/LAB2/tripdata_2017-02.csv", header=0)
data = pd.concat([data1,data2])
print (data.columns)

#We select the columns corresponding to the fare, tip, tolls and extra amount and the improvement surcharge, excluding the taxes and we add them together.

no_taxes = data['fare_amount'] + data['tip_amount'] + data['tolls_amount'] + data['extra'] + data['improvement_surcharge']

#Now, we compute the mean of this quantity. 
meanprice=no_taxes.mean()
print (meanprice, 'No taxes mean')

#Let's compare this mean value to the mean of the total amount given to the driver.

total = data['total_amount']
total_mean = total.mean()
print (total_mean, 'Total amount mean')

#We can see that there is a difference of close to 0.5$ between these amounts, meaning that the fraction of the total amount spent in taxes
#is close to 0.5$ for every trip. Let's check this fact: 
print(data['mta_tax'].mean())

end = time.time()

print (end-start)
#We can check how the amount paid in taxes varies in terms of the 
#snd_analysis = data.loc[:,['payment_type', 'mta_tax', 'total_amount']]
#snd_analysis.groupby(["payment_type"])["mta_tax"].plot(kind='density',xlim=[0,1],xlabel="Price")


Index(['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'passenger_count', 'trip_distance', 'RatecodeID', 'store_and_fwd_flag',
       'PULocationID', 'DOLocationID', 'payment_type', 'fare_amount', 'extra',
       'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge',
       'total_amount'],
      dtype='object')
15.353715345033166 No taxes mean
15.854891789885082 Total amount mean
0.4974637874404095
5.099730730056763


In [1]:
#!pip install pyspark
import findspark 
findspark.init()
from pyspark.sql import SparkSession
import pandas as pd 
spark = SparkSession.builder.appName("prueba").master("local[*]").getOrCreate()

data_spark = spark.read.option("header","true").csv("DATA/*.csv")

#Now, we select the columns corresponding to the amount of money excluding the taxes. 
no_taxes_rdd = data_spark.select(['fare_amount', 'tip_amount', 'tolls_amount', 'extra', 'improvement_surcharge'])

#Since we want to compute the mean value, we need to compute the sum of all the elements of the object, the count of these elements 
#and divide them. 
#Let's start by computing and printing the number of observations present in the rdd object. 
count = no_taxes_rdd.count()
print(count)
#Secondly, we sum the elements of the rdd thanks to the function .reduce(), which allows us to make computations between the elements
#of a list. 
suma = no_taxes_rdd.reduce(lambda a,b : (a+b))
print(suma)
#Lastly, we divide them and obtain the mean value. 
print(suma/count)

#We can do the same thing for the 'mta_tax' value. 

taxes = data_spark.select(['mta_tax'])
count2 = taxes.count()
suma2 = taxes.reduce(lambda a,b : a+b)

mean = suma2/count2
print(mean)

#We have obtained the same results as we did in the python version, obtaining a mean value for the taxes amount close to 0.5$. 

spark.stop()

ModuleNotFoundError: No module named 'findspark'